In [40]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras

RANDOM_SEED = 42

# Specify each path

In [41]:
dataset = 'data/keypoint.csv'
model_save_path = 'model/keypoint_classifier.keras'
tflite_save_path = 'model/keypoint_classifier.tflite'

# Set number of classes

In [42]:
NUM_CLASSES = 8

# Dataset reading

In [43]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [44]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [46]:
model = keras.models.Sequential([
    keras.layers.Input((21 * 2, )),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [47]:
model.summary()  

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_10 (Dropout)            │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 8)              │            88 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,158 (4.52 KB)

 Trainable params: 1,158 (4.52 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# Model checkpoint callback
cp_callback = keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [49]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [50]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.1953 - loss: 2.0504
Epoch 1: saving model to model/keypoint_classifier.keras
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.1540 - loss: 2.0775 - val_accuracy: 0.3200 - val_loss: 1.9876
Epoch 2/1000
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.2188 - loss: 2.0042
Epoch 2: saving model to model/keypoint_classifier.keras
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2147 - loss: 2.0107 - val_accuracy: 0.3767 - val_loss: 1.9372
Epoch 3/1000
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1641 - loss: 2.0079
Epoch 3: saving model to model/keypoint_classifier.keras
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2399 - loss: 1.9688 - val_accuracy: 0.3900 - val_loss: 1.8952
Epoch 4/1000
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.2891 - loss: 1.8978
Epoch 4: saving model to model/keypoint_classifier.keras
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2820 - loss: 1.9091 - val_

In [51]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9746 - loss: 0.2652 


In [52]:
# Loading the saved model
model = keras.models.load_model(model_save_path)

In [53]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
[0.00995189 0.01812687 0.02161474 0.00463266 0.12420244 0.00132284
 0.69174325 0.12840529]
6


# Confusion matrix

In [54]:
from sklearn.metrics import classification_report
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
print('Classification Report')
print(classification_report(y_test, y_pred))

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        79
           1       0.98      1.00      0.99        80
           2       0.99      0.99      0.99        74
           3       0.97      1.00      0.99        70
           4       0.96      1.00      0.98        82
           5       1.00      0.90      0.95        73
           6       0.95      0.97      0.96        71
           7       1.00      0.92      0.96        71

    accuracy                           0.97       600
   macro avg       0.97      0.97      0.97       600
weighted avg       0.97      0.97      0.97       600



# Convert to model for Tensorflow-Lite

In [55]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [56]:
# Convert the Keras model to a ConcreteFunction
run_model = tf.function(lambda x: model(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

In [57]:
# Transform model (quantization)
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

6588

# Inference test

In [58]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [59]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [60]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [61]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: total: 0 ns
Wall time: 456 µs


In [62]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[0.00995189 0.01812688 0.02161475 0.00463267 0.12420247 0.00132284
 0.69174314 0.12840529]
6
